In [8]:
from pathlib import Path

# Path ke direktori YOLOv5
yolov5_dir = Path('../../yolov5')

# Print struktur folder
def print_folder_structure(path, indent=0):
    if path.is_dir():
        print('  ' * indent + f'📂 {path.name}')
        for item in path.iterdir():
            print_folder_structure(item, indent + 1)
    else:
        print('  ' * indent + f'📄 {path.name}')

# Cek apakah direktori yolov5 ada
if yolov5_dir.exists() and yolov5_dir.is_dir():
    print(f"Folder 'yolov5' ditemukan di {yolov5_dir.resolve()}")
    print("Struktur foldernya:")
    print_folder_structure(yolov5_dir)
else:
    print(f"Folder 'yolov5' tidak ditemukan di {yolov5_dir.resolve()}")


Folder 'yolov5' ditemukan di C:\Users\gabri\Documents\yolov5
Struktur foldernya:
📂 yolov5
  📄 .dockerignore
  📂 .git
    📄 config
    📄 description
    📄 HEAD
    📂 hooks
      📄 applypatch-msg.sample
      📄 commit-msg.sample
      📄 fsmonitor-watchman.sample
      📄 post-update.sample
      📄 pre-applypatch.sample
      📄 pre-commit.sample
      📄 pre-merge-commit.sample
      📄 pre-push.sample
      📄 pre-rebase.sample
      📄 pre-receive.sample
      📄 prepare-commit-msg.sample
      📄 push-to-checkout.sample
      📄 sendemail-validate.sample
      📄 update.sample
    📄 index
    📂 info
      📄 exclude
    📂 logs
      📄 HEAD
      📂 refs
        📂 heads
          📄 master
        📂 remotes
          📂 origin
            📄 HEAD
    📂 objects
      📂 info
      📂 pack
        📄 pack-f2f88cb4cdb1081a0be75dfe65a8a77cbd8ceeb3.idx
        📄 pack-f2f88cb4cdb1081a0be75dfe65a8a77cbd8ceeb3.pack
        📄 pack-f2f88cb4cdb1081a0be75dfe65a8a77cbd8ceeb3.rev
    📄 packed-refs
    📂 refs
      📂 

In [9]:
import sys
from pathlib import Path

yolov5_dir = Path('../../yolov5')
sys.path.append(str(yolov5_dir))

# Sekarang Anda bisa mengimpor modul YOLOv5
from models.common import DetectMultiBackend


Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\gabri\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [13]:
from models.common import DetectMultiBackend
model = DetectMultiBackend(yolov5_dir / 'yolov5s.pt', device='cpu')  # Ganti 'cpu' dengan 'cuda' jika menggunakan GPU


Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs


In [20]:
import os
import random
from PIL import Image
import matplotlib.pyplot as plt
import torch

# Path ke folder dataset utama
main_directory = '../dataset_padang_food'

# Memilih folder dan gambar secara acak dari dataset
folders = [f for f in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, f))]
random_folder = random.choice(folders)
random_folder_path = os.path.join(main_directory, random_folder)
image_files = [f for f in os.listdir(random_folder_path) if f.endswith(('jpg', 'jpeg', 'png'))]
random_image_file = random.choice(image_files)
random_image_path = os.path.join(random_folder_path, random_image_file)

In [21]:
# Memuat dan menampilkan gambar asli
image = Image.open(random_image_path)
plt.imshow(image)
plt.title("Gambar Asli")
plt.axis('off')
plt.show()

# Melakukan deteksi objek menggunakan YOLOv5 (tanpa size argument)
results = model(random_image_path)  # Menggunakan model langsung untuk prediksi tanpa size argument

# Menampilkan hasil deteksi
results.show()  # Menampilkan hasil deteksi

AttributeError: 'str' object has no attribute 'shape'

In [26]:
import sys
from pathlib import Path

yolov5_dir = Path('../../yolov5')
sys.path.append(str(yolov5_dir))

# Import modul yang diperlukan dari YOLOv5
from models.common import DetectMultiBackend
from utils.augmentations import letterbox
from utils.general import non_max_suppression, scale_boxes
from utils.plots import colors, plot_one_box
import torch

# Inisialisasi model YOLOv5
model = DetectMultiBackend(yolov5_dir / 'yolov5s.pt', device='cpu')  # Ganti 'cpu' dengan 'cuda' jika menggunakan GPU
model.names = model.names  # Mendapatkan nama kelas

import os
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Path ke folder dataset utama
main_directory = '../dataset_padang_food'

# Memilih folder dan gambar secara acak dari dataset
folders = [f for f in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, f))]
random_folder = random.choice(folders)
random_folder_path = os.path.join(main_directory, random_folder)
image_files = [f for f in os.listdir(random_folder_path) if f.endswith(('jpg', 'jpeg', 'png'))]
random_image_file = random.choice(image_files)
random_image_path = os.path.join(random_folder_path, random_image_file)

# Membaca gambar menggunakan OpenCV
img0 = cv2.imread(random_image_path)  # BGR format

# Preprocessing gambar dengan letterbox untuk menjaga aspek rasio
img, ratio, (dw, dh) = letterbox(img0, 640, stride=32, auto=True)

# Mengonversi gambar ke tensor
img = img.transpose((2, 0, 1))  # HWC ke CHW
img = np.ascontiguousarray(img)
img = torch.from_numpy(img).to('cpu')  # Ganti 'cpu' dengan 'cuda' jika menggunakan GPU
img = img.float()  # Mengonversi ke float32
img /= 255.0  # Normalisasi ke 0.0 - 1.0

if img.ndimension() == 3:
    img = img.unsqueeze(0)

# Inferensi dengan model
with torch.no_grad():
    pred = model(img, augment=False, visualize=False)

# Non-Max Suppression untuk menghilangkan duplikasi deteksi
pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45, classes=None, agnostic=False)

# Memproses deteksi dan menampilkan hasil
for i, det in enumerate(pred):  # Deteksi per gambar
    im0 = img0.copy()

    if len(det):
        # Menskalakan kotak deteksi ke ukuran gambar asli
        det[:, :4] = scale_boxes(img.shape[2:], det[:, :4], im0.shape).round()

        # Menggambar kotak deteksi dan label pada gambar
        for *xyxy, conf, cls in reversed(det):
            label = f'{model.names[int(cls)]} {conf:.2f}'
            plot_one_box(xyxy, im0, label=label, color=colors(int(cls), True), line_thickness=2)

# Menampilkan gambar dengan deteksi
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(im0, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()


ImportError: cannot import name 'plot_one_box' from 'utils.plots' (C:\Users\gabri\Documents\NutritionCalculate-ComputerVision\base\..\..\yolov5\utils\plots.py)

In [27]:
import sys
from pathlib import Path

yolov5_dir = Path('../../yolov5')
sys.path.append(str(yolov5_dir))

# Import modul yang diperlukan dari YOLOv5
from models.common import DetectMultiBackend
from utils.augmentations import letterbox
from utils.general import non_max_suppression, scale_boxes
import torch

# Definisi fungsi plot_one_box yang menggantikan error import sebelumnya
def plot_one_box(xyxy, img, color=(255, 0, 0), label=None, line_thickness=3):
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1  # Ketebalan garis
    c1, c2 = (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    if label:
        tf = max(tl - 1, 1)  # Ketebalan font
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA)  # Mengisi background label
        cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

# Inisialisasi model YOLOv5
model = DetectMultiBackend(yolov5_dir / 'yolov5s.pt', device='cpu')  # Ganti 'cpu' dengan 'cuda' jika menggunakan GPU
model.names = model.names  # Mendapatkan nama kelas

import os
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Path ke folder dataset utama
main_directory = '../dataset_padang_food'

# Memilih folder dan gambar secara acak dari dataset
folders = [f for f in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, f))]
random_folder = random.choice(folders)
random_folder_path = os.path.join(main_directory, random_folder)
image_files = [f for f in os.listdir(random_folder_path) if f.endswith(('jpg', 'jpeg', 'png'))]
random_image_file = random.choice(image_files)
random_image_path = os.path.join(random_folder_path, random_image_file)

# Membaca gambar menggunakan OpenCV
img0 = cv2.imread(random_image_path)  # BGR format

# Preprocessing gambar dengan letterbox untuk menjaga aspek rasio
img, ratio, (dw, dh) = letterbox(img0, 640, stride=32, auto=True)

# Mengonversi gambar ke tensor
img = img.transpose((2, 0, 1))  # HWC ke CHW
img = np.ascontiguousarray(img)
img = torch.from_numpy(img).to('cpu')  # Ganti 'cpu' dengan 'cuda' jika menggunakan GPU
img = img.float()  # Mengonversi ke float32
img /= 255.0  # Normalisasi ke 0.0 - 1.0

if img.ndimension() == 3:
    img = img.unsqueeze(0)

# Inferensi dengan model
with torch.no_grad():
    pred = model(img, augment=False, visualize=False)

# Non-Max Suppression untuk menghilangkan duplikasi deteksi
pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45, classes=None, agnostic=False)

# Memproses deteksi dan menampilkan hasil
for i, det in enumerate(pred):  # Deteksi per gambar
    im0 = img0.copy()

    if len(det):
        # Menskalakan kotak deteksi ke ukuran gambar asli
        det[:, :4] = scale_boxes(img.shape[2:], det[:, :4], im0.shape).round()

        # Menggambar kotak deteksi dan label pada gambar
        for *xyxy, conf, cls in reversed(det):
            label = f'{model.names[int(cls)]} {conf:.2f}'
            plot_one_box(xyxy, im0, label=label, color=(255, 0, 0), line_thickness=2)

# Menampilkan gambar dengan deteksi
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(im0, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()


Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs


In [28]:
import sys
from pathlib import Path

yolov5_dir = Path('../../yolov5')
sys.path.append(str(yolov5_dir))

# Import modul yang diperlukan dari YOLOv5
from models.common import DetectMultiBackend
from utils.augmentations import letterbox
from utils.general import non_max_suppression, scale_boxes
import torch

# Definisi fungsi plot_one_box yang menggantikan error import sebelumnya
def plot_one_box(xyxy, img, color=(255, 0, 0), label=None, line_thickness=3):
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1  # Ketebalan garis
    c1, c2 = (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    if label:
        tf = max(tl - 1, 1)  # Ketebalan font
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA)  # Mengisi background label
        cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

# Inisialisasi model YOLOv5
model = DetectMultiBackend(yolov5_dir / 'yolov5s.pt', device='cpu')  # Ganti 'cpu' dengan 'cuda' jika menggunakan GPU
model.names = model.names  # Mendapatkan nama kelas

import os
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Path ke folder dataset utama
main_directory = '../dataset_padang_food'

# Memilih folder dan gambar secara acak dari dataset
folders = [f for f in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, f))]
random_folder = random.choice(folders)
random_folder_path = os.path.join(main_directory, random_folder)
image_files = [f for f in os.listdir(random_folder_path) if f.endswith(('jpg', 'jpeg', 'png'))]
random_image_file = random.choice(image_files)
random_image_path = os.path.join(random_folder_path, random_image_file)

# Cetak gambar yang dipilih
print(f"Gambar yang dipilih: {random_image_path}")

# Membaca gambar menggunakan OpenCV
img0 = cv2.imread(random_image_path)  # BGR format

# Preprocessing gambar dengan letterbox untuk menjaga aspek rasio
img, ratio, (dw, dh) = letterbox(img0, 640, stride=32, auto=True)

# Mengonversi gambar ke tensor
img = img.transpose((2, 0, 1))  # HWC ke CHW
img = np.ascontiguousarray(img)
img = torch.from_numpy(img).to('cpu')  # Ganti 'cpu' dengan 'cuda' jika menggunakan GPU
img = img.float()  # Mengonversi ke float32
img /= 255.0  # Normalisasi ke 0.0 - 1.0

if img.ndimension() == 3:
    img = img.unsqueeze(0)

# Inferensi dengan model
with torch.no_grad():
    pred = model(img, augment=False, visualize=False)

# Cetak hasil prediksi mentah
print(f"Hasil prediksi mentah: {pred}")

# Non-Max Suppression untuk menghilangkan duplikasi deteksi
pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45, classes=None, agnostic=False)

# Memproses deteksi dan menampilkan hasil
for i, det in enumerate(pred):  # Deteksi per gambar
    im0 = img0.copy()

    if len(det):
        # Menskalakan kotak deteksi ke ukuran gambar asli
        det[:, :4] = scale_boxes(img.shape[2:], det[:, :4], im0.shape).round()

        # Menggambar kotak deteksi dan label pada gambar
        for *xyxy, conf, cls in reversed(det):
            label = f'{model.names[int(cls)]} {conf:.2f}'
            plot_one_box(xyxy, im0, label=label, color=(255, 0, 0), line_thickness=2)
    else:
        print("Tidak ada deteksi pada gambar ini.")

# Menampilkan gambar dengan deteksi
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(im0, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()


Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs


Gambar yang dipilih: ../dataset_padang_food\ayam_pop\ayam_pop (36).jpg
Hasil prediksi mentah: [tensor([[[4.24220e+00, 3.67757e+00, 9.21342e+00,  ..., 1.44761e-03, 6.77086e-04, 3.34880e-03],
         [1.28979e+01, 4.84432e+00, 2.57964e+01,  ..., 1.16218e-03, 8.75519e-04, 2.78581e-03],
         [1.81673e+01, 5.04127e+00, 3.38297e+01,  ..., 1.23018e-03, 9.23986e-04, 3.30452e-03],
         ...,
         [5.65937e+02, 4.24526e+02, 1.46989e+02,  ..., 2.87458e-03, 8.57689e-04, 1.02618e-03],
         [5.84386e+02, 4.27810e+02, 1.37772e+02,  ..., 2.98620e-03, 8.53880e-04, 9.99433e-04],
         [6.20809e+02, 4.33285e+02, 1.33838e+02,  ..., 2.42703e-03, 9.58183e-04, 1.06796e-03]]]), [tensor([[[[[ 6.05696e-02, -8.06512e-02, -8.03119e-02,  ..., -6.53639e+00, -7.29703e+00, -5.69580e+00],
           [ 2.25422e-01,  2.11869e-01,  1.40555e+00,  ..., -6.75630e+00, -7.03982e+00, -5.88043e+00],
           [-4.66455e-01,  2.61804e-01,  2.43750e+00,  ..., -6.69936e+00, -6.98589e+00, -5.70915e+00],
        

In [30]:
import sys
from pathlib import Path

yolov5_dir = Path('../../yolov5')
sys.path.append(str(yolov5_dir))

# Import modul yang diperlukan dari YOLOv5
from models.common import DetectMultiBackend
from utils.augmentations import letterbox
from utils.general import non_max_suppression, scale_boxes
import torch

# Definisi fungsi plot_one_box yang menggantikan error import sebelumnya
def plot_one_box(xyxy, img, color=(255, 0, 0), label=None, line_thickness=3):
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1  # Ketebalan garis
    c1, c2 = (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    if label:
        tf = max(tl - 1, 1)  # Ketebalan font
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA)  # Mengisi background label
        cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

# Inisialisasi model YOLOv5
model = DetectMultiBackend(yolov5_dir / 'yolov5s.pt', device='cpu')  # Ganti 'cpu' dengan 'cuda' jika menggunakan GPU
model.names = model.names  # Mendapatkan nama kelas

import os
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Path ke folder dataset utama
main_directory = '../dataset_padang_food'

# Memilih folder dan gambar secara acak dari dataset
folders = [f for f in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, f))]
random_folder = random.choice(folders)
random_folder_path = os.path.join(main_directory, random_folder)
image_files = [f for f in os.listdir(random_folder_path) if f.endswith(('jpg', 'jpeg', 'png'))]
random_image_file = random.choice(image_files)
random_image_path = os.path.join(random_folder_path, random_image_file)

# Cetak gambar yang dipilih
print(f"Gambar yang dipilih: {random_image_path}")

# Membaca gambar menggunakan OpenCV
img0 = cv2.imread(random_image_path)  # BGR format

# Preprocessing gambar dengan letterbox untuk menjaga aspek rasio
img, ratio, (dw, dh) = letterbox(img0, 640, stride=32, auto=True)

# Mengonversi gambar ke tensor
img = img.transpose((2, 0, 1))  # HWC ke CHW
img = np.ascontiguousarray(img)
img = torch.from_numpy(img).to('cpu')  # Ganti 'cpu' dengan 'cuda' jika menggunakan GPU
img = img.float()  # Mengonversi ke float32
img /= 255.0  # Normalisasi ke 0.0 - 1.0

if img.ndimension() == 3:
    img = img.unsqueeze(0)

# Inferensi dengan model
with torch.no_grad():
    pred = model(img, augment=False, visualize=False)

# Cetak hasil prediksi mentah
print(f"Hasil prediksi mentah: {pred}")

# Non-Max Suppression untuk menghilangkan duplikasi deteksi
pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45, classes=None, agnostic=False)

# Cetak hasil setelah NMS
print(f"Hasil setelah NMS: {pred}")

# Memproses deteksi dan menampilkan hasil
for i, det in enumerate(pred):  # Deteksi per gambar
    im0 = img0.copy()

    if len(det):
        print(f"Deteksi ditemukan: {det}")
        # Menskalakan kotak deteksi ke ukuran gambar asli
        det[:, :4] = scale_boxes(img.shape[2:], det[:, :4], im0.shape).round()

        # Menggambar kotak deteksi dan label pada gambar
        for *xyxy, conf, cls in reversed(det):
            label = f'{model.names[int(cls)]} {conf:.2f}'
            plot_one_box(xyxy, im0, label=label, color=(255, 0, 0), line_thickness=2)
    else:
        print("Tidak ada deteksi pada gambar ini.")

# Menampilkan gambar dengan deteksi
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(im0, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()


Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs


Gambar yang dipilih: ../dataset_padang_food\telur_balado\telur_balado (98).jpg
Hasil prediksi mentah: [tensor([[[4.35311e+00, 4.33263e+00, 8.56167e+00,  ..., 1.29694e-03, 4.00162e-04, 5.53477e-03],
         [1.30101e+01, 3.49128e+00, 2.49357e+01,  ..., 9.81133e-04, 5.56928e-04, 5.28561e-03],
         [1.78839e+01, 2.88383e+00, 3.22798e+01,  ..., 1.04120e-03, 4.96898e-04, 5.55464e-03],
         ...,
         [5.54109e+02, 3.83120e+02, 1.79515e+02,  ..., 1.84940e-03, 7.79028e-04, 1.72810e-03],
         [5.88739e+02, 3.89545e+02, 1.49464e+02,  ..., 1.78974e-03, 9.66039e-04, 2.51022e-03],
         [6.20806e+02, 3.98454e+02, 1.46934e+02,  ..., 1.99958e-03, 8.89538e-04, 2.22064e-03]]]), [tensor([[[[[ 8.83361e-02,  8.32055e-02, -1.49693e-01,  ..., -6.64645e+00, -7.82324e+00, -5.19115e+00],
           [ 2.53890e-01, -1.27351e-01,  1.32222e+00,  ..., -6.92582e+00, -7.49252e+00, -5.23747e+00],
           [-5.41901e-01, -2.80875e-01,  2.17879e+00,  ..., -6.86634e+00, -7.60663e+00, -5.18755e+00],


In [31]:
import sys
from pathlib import Path

yolov5_dir = Path('../../yolov5')
sys.path.append(str(yolov5_dir))

# Import modul yang diperlukan dari YOLOv5
from models.common import DetectMultiBackend
from utils.augmentations import letterbox
from utils.general import non_max_suppression, scale_boxes
import torch

# Definisi fungsi plot_one_box yang menggantikan error import sebelumnya
def plot_one_box(xyxy, img, color=(255, 0, 0), label=None, line_thickness=3):
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1  # Ketebalan garis
    c1, c2 = (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    if label:
        tf = max(tl - 1, 1)  # Ketebalan font
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA)  # Mengisi background label
        cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

# Inisialisasi model YOLOv5
model = DetectMultiBackend(yolov5_dir / 'yolov5s.pt', device='cpu')  # Ganti 'cpu' dengan 'cuda' jika menggunakan GPU
model.names = model.names  # Mendapatkan nama kelas

import os
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Path ke folder dataset utama
main_directory = '../dataset_padang_food'

# Memilih folder dan gambar secara acak dari dataset
folders = [f for f in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, f))]
random_folder = random.choice(folders)
random_folder_path = os.path.join(main_directory, random_folder)
image_files = [f for f in os.listdir(random_folder_path) if f.endswith(('jpg', 'jpeg', 'png'))]
random_image_file = random.choice(image_files)
random_image_path = os.path.join(random_folder_path, random_image_file)

# Cetak gambar yang dipilih
print(f"Gambar yang dipilih: {random_image_path}")

# Membaca gambar menggunakan OpenCV
img0 = cv2.imread(random_image_path)  # BGR format

# Preprocessing gambar dengan letterbox untuk menjaga aspek rasio
img, ratio, (dw, dh) = letterbox(img0, 640, stride=32, auto=True)

# Mengonversi gambar ke tensor
img = img.transpose((2, 0, 1))  # HWC ke CHW
img = np.ascontiguousarray(img)
img = torch.from_numpy(img).to('cpu')  # Ganti 'cpu' dengan 'cuda' jika menggunakan GPU
img = img.float()  # Mengonversi ke float32
img /= 255.0  # Normalisasi ke 0.0 - 1.0

if img.ndimension() == 3:
    img = img.unsqueeze(0)

# Inferensi dengan model
with torch.no_grad():
    pred = model(img, augment=False, visualize=False)

# Cetak hasil prediksi mentah
print(f"Hasil prediksi mentah: {pred}")

# Non-Max Suppression untuk menghilangkan duplikasi deteksi
pred = non_max_suppression(pred, conf_thres=0.25, iou_thres=0.45, classes=None, agnostic=False)

# Cetak hasil setelah NMS
print(f"Hasil setelah NMS: {pred}")

# Memproses deteksi dan menampilkan hasil
top_predictions = []

for i, det in enumerate(pred):  # Deteksi per gambar
    im0 = img0.copy()

    if len(det):
        print(f"Deteksi ditemukan: {det}")
        # Menskalakan kotak deteksi ke ukuran gambar asli
        det[:, :4] = scale_boxes(img.shape[2:], det[:, :4], im0.shape).round()

        # Menggambar kotak deteksi dan label pada gambar
        for *xyxy, conf, cls in reversed(det):
            label = f'{model.names[int(cls)]} {conf:.2f}'
            plot_one_box(xyxy, im0, label=label, color=(255, 0, 0), line_thickness=2)
            top_predictions.append((model.names[int(cls)], conf.item()))

    else:
        print("Tidak ada deteksi pada gambar ini.")

# Menampilkan gambar dengan deteksi
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(im0, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title(f'Prediction: {top_predictions[0][0]}, Confidence: {top_predictions[0][1]:.4f}')
plt.show()

# Menampilkan top 3 prediksi
print("\nTop 3 Predictions:")
top_predictions = sorted(top_predictions, key=lambda x: x[1], reverse=True)[:3]
for pred_label, pred_conf in top_predictions:
    print(f"{pred_label}: {pred_conf:.4f}")


Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs


Gambar yang dipilih: ../dataset_padang_food\gulai_ikan\gulai_ikan (108).jpg
Hasil prediksi mentah: [tensor([[[3.58706e+00, 1.93892e+00, 7.45721e+00,  ..., 1.11769e-03, 9.06252e-04, 6.99056e-03],
         [1.24546e+01, 2.00475e+00, 2.51840e+01,  ..., 8.42291e-04, 1.10678e-03, 6.30401e-03],
         [1.80794e+01, 1.86019e+00, 3.21575e+01,  ..., 8.79108e-04, 1.24778e-03, 8.13316e-03],
         ...,
         [5.59298e+02, 5.40782e+02, 1.51945e+02,  ..., 1.48204e-03, 1.05458e-03, 1.45297e-03],
         [5.90408e+02, 5.35355e+02, 1.03348e+02,  ..., 1.01981e-03, 8.87611e-04, 1.68869e-03],
         [6.21596e+02, 5.46713e+02, 1.14985e+02,  ..., 9.81291e-04, 9.26997e-04, 1.59037e-03]]]), [tensor([[[[[-1.03327e-01, -5.27148e-01, -2.74610e-01,  ..., -6.79537e+00, -7.00529e+00, -4.95618e+00],
           [ 1.13773e-01, -5.09560e-01,  1.34599e+00,  ..., -7.07854e+00, -6.80519e+00, -5.06025e+00],
           [-4.89694e-01, -5.48288e-01,  2.16029e+00,  ..., -7.03572e+00, -6.68514e+00, -4.80364e+00],
   

IndexError: list index out of range

In [32]:
import os
import random
from PIL import Image
import matplotlib.pyplot as plt
import cv2  # Menggunakan OpenCV untuk memuat gambar
import torch

# Path ke folder dataset utama
main_directory = '../dataset_padang_food'

# Memilih folder dan gambar secara acak dari dataset
folders = [f for f in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, f))]
random_folder = random.choice(folders)
random_folder_path = os.path.join(main_directory, random_folder)
image_files = [f for f in os.listdir(random_folder_path) if f.endswith(('jpg', 'jpeg', 'png'))]
random_image_file = random.choice(image_files)
random_image_path = os.path.join(random_folder_path, random_image_file)

# Memuat dan menampilkan gambar asli menggunakan PIL untuk visualisasi
image_pil = Image.open(random_image_path)
plt.imshow(image_pil)
plt.title("Gambar Asli")
plt.axis('off')
plt.show()

# Memuat gambar dengan OpenCV (gambar sebagai array numpy)
image_cv2 = cv2.imread(random_image_path)

# Melakukan deteksi objek menggunakan YOLOv5
results = model(image_cv2)  # Melakukan deteksi pada gambar yang dimuat dengan OpenCV

# Menampilkan hasil deteksi
results.show()  # Menampilkan hasil deteksi (atau bisa juga menggunakan results.save())

# Mendapatkan hasil deteksi
boxes = results.xyxy[0].numpy()  # Mendapatkan bounding box dalam format (xmin, ymin, xmax, ymax, confidence, class)
print(boxes)  # Print hasil deteksi sebagai array numpy


ValueError: not enough values to unpack (expected 4, got 3)